# 基于支持向量机的Digits手写数字识别

# 1、加载查看Digits数据集

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve,auc
from sklearn.multiclass import OneVsRestClassifier
import numpy as np
from scipy import interp
from itertools import cycle


# 加载Digits数据集
digits = load_digits()
data = digits.data # 样本数据
target = digits.target # 标签数据
print(data.shape, target.shape) # 输出数组形状

# 使用Matplotlib，显示前20张图片
fig = plt.figure(figsize=(8, 8), facecolor='w')
for i in range(20):
    ax = fig.add_subplot(4, 5, i+1) 
    
    # matshow方法将像素矩阵显示为图片
    # data中的图片像素为长度64的一维数组，需要转成8*8的二维数组来显示
'''
请使用matshow方法将像素矩阵显示为图片
'''
    
    
plt.show()

# 2、拆分数据集、创建模型
     

    1.SVC可选的核函数参数有很多，请查看SVC函数的文档，学习不同参数，使用不同的核函数建立模型并进行实验。
        |___    kernel : {'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'} or callable,
                default='rbf'
                Specifies the kernel type to be used in the algorithm.
                If none is given, 'rbf' will be used. If a callable is given it is
                used to pre-compute the kernel matrix from data matrices; that matrix
                should be an array of shape ``(n_samples, n_samples)``.
    
    2.这里使用python字典配合循环块批量定义不同参数的SVC模型，以精简代码，便于后续改进工作的进行;
    也可以定义完模型后重复编写代码调用模型进行拟合，这样使用更为简单直接，但复用性不高。
        |___    e.g.    model = {}
                    model['key1'] = SVC("parameter1")
                    model['key2'] = SVC("parameter2")
                    for key ,value in modelss.items():
                         model = value.fit(X_train, y_train)
                         score = model.score(X_test, y_test)
                         print("%s score is : %.2f" %(key, score))
                         y_pred = model.predict(X_test[:20])
                         print('%s验证的预测数字：'%key, y_pred)
                         print('%s验证的实际数字：'%key, y_test[:20])
                         print("\n")


In [ ]:
# 拆分数据集
X_train, X_test, y_train, y_test = train_test_split(data, target, 
                                                    test_size=0.7, random_state=0)
print(X_train.shape, X_test.shape)


#定义模型
'''
SVC的gamma参数已给出，现需要按照相应的字典名设置kernel函数为'linear', 'poly', 'rbf', 'sigmoid'
'''







#训练模型并输出结果
for key ,value in modelss.items():
    model = value.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    print("%s score is : %.2f" %(key, score))
    y_pred = model.predict(X_test[:20])
    print('%s验证的预测数字：'%key, y_pred)
    print('%s验证的实际数字：'%key, y_test[:20])
    print("\n")

# 3.1、正则化参数的修改

In [ ]:
'''
请使用四个正则化参数0.01, 0.05, 0.1, 0.5,列表记录四组分类正确率的数据（请修改模型中的C值）
'''
# 例如：
# modelss = {}
# modelss["rbf_kernel"] = SVC(C=1,kernel = 'rbf',gamma = 'scale')
# modelss["poly_kernel"] = SVC(C=1,kernel = 'poly',gamma = 'scale')
# modelss["sigmoid_kernel"] = SVC(C=1,kernel = 'sigmoid',gamma = 'scale')
# modelss["linear_kernel"]=  SVC(C=1,kernel = 'linear',gamma = 'scale')

#训练模型并输出结果
for key ,value in modelss.items():
    model = value.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    print("%s score is : %.2f" %(key, score))
    y_pred = model.predict(X_test[:50])
    print('%s验证的预测数字：'%key, y_pred)
    print('%s验证的实际数字：'%key, y_test[:50])
    print("\n")
    
    

# 3.2、多项式维度参数degree的修改

In [ ]:
#请修改SVC()中的degree参数分别为 1, 3, 5, 7, 9 ，列表记录模型准确率
model = SVC(kernel='poly',degree=9)
model = model.fit(X_train, y_train)
score = model.score(X_test, y_test)
print("%s score is : %.2f" %(key, score))
y_pred = model.predict(X_test[:50])
print('%s验证的预测数字：'%key, y_pred)
print('%s验证的实际数字：'%key, y_test[:50])
print("\n")

# 4、绘制ROC曲线

In [ ]:
y = label_binarize(target, classes = [0,1,2,3,4,5,6,7,8,9])
n_classes = y.shape[1]

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=.5,random_state=0)

classifier = OneVsRestClassifier(SVC(kernel='linear', probability=True,
                                 random_state=1))
y_score = classifier.fit(X_train, y_train).decision_function(X_test)

#计算每一类的ROC
'''此部分循环计算了每一类分类相对于其它所有类别的真阳率tpr和假阳率fpr，并记录在响应字典值中'''
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:,i], y_score[:,i])
    roc_auc[i] = auc(fpr[i], tpr[i])


#绘制ROC曲线
plt.figure()
lw = 2
'''
下列示例中绘制的是数字0的ROC曲线，上一部分代码已经计算了每一个分类相对于其他所有分类
的真阳率tpr和假阳率fpr；请改变参数值，使用不同颜色绘制数字8和9相应的两条ROC曲线
'''
#plt.plot(fpr[0], tpr[0], color='darkorange', lw=lw, label='ROC curve 0(area = %0.2f)' % roc_auc[0])
'''
//=============================请在此编写代码=======================//
'''

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC CURVE')
plt.legend(loc="lower right")
plt.show()
